In [2]:
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools
cache=functools.lru_cache(10**6)

In [18]:
class Game:
  def actions(Self, state):
    raise NotImplementedError

  def result (Self, state, move):
    raise NotImplementedError

  def is_terminal(Self, state):
    return not self.actions(state)

  def utility(self, state, player):
    raise NotImplementedError

In [19]:
def play_game(game, strategies: dict, verbose=False):
  state = game.intial
  while not game.is_terminal(state):
    player = state.to_move
    move = strategies[player](game, state)
    state = game.result(state,move)
    if verbose:
      print('Player', player, 'move:', move)
      print(state)
    return state

In [20]:
def minimax_search(game, state):
  player = state.to_move
  def max_value(state):
    if game.is_terminal(state):
      return game.utility(state, player), None
    v, move = -infinity, None
    for a in game.actions(state):
      v2, _ = min_value(game.result(state, a))
      if v2 > v:
        v, move = v2, a
    return v, move
def min_value(state):
  if game.is_terminal(state):
    return game.utility(state, Iplayer), None
  v, move = +infinity, None
  for a in game.actions(state):
    v2, _ = max_value(game.result(state, a))
    if v2 < v:
      v, move = v2, a
  return v, move

In [27]:
class tictactoe(Game):
  def _init_(self, height=3, width=3, k=3):
    self.k = k
    self.squares = {(x, y) for x in range(width) for y in range(height)}
    self.initial = Board(height=height, width=width, to_move='X', utility=0)
  def actions(self, board):
    return self.squares - set(board)
  def result(self, board, square):
      player = board.to_move
      board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
      win = k_in_row(board, player, square, self.k)
      board.utility = (0 if not win else +1 if player == 'X' else -1)
      return board
  def utility(self, board, player):
    return board.utility if player == 'X' else -board.utility
  def is_terminal(self, board):
    return board.utility != 0 or len(self.squares) == len(board)
  def display(self, board):
      print(board)
  def k_in_row(board, player, square, k):
    def in_row(x, y, dx, dy):
      return 0 if board.get((x, y)) != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) >= k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

In [28]:
class Board(defaultdict):
  empty = '.'
  off = '#'
  def _init_(self, width=8, height=8, to_move=None, **kwds):
    self._dict_.update(width=width, height=height, to_move=to_move, **kwds)
  def new(self, changes: dict, **kwds) -> 'Board':
    board = Board(width=self.width, height=self.height, **kwds)
    board.update(self)
    board.update(changes)
    return board
  def _missing_(self, loc):
    x, y = loc
    if 0 <= x < self.width and 0 <= y < self.height:
      return self.empty
    else:
      return self.off
  def _hash_(self):
    return hash(tuple(sorted(self.items()))) + hash(self.to_move)
  def _repr_(self):
    def row(y): return ' '.join(self[x, y] for x in range(self.width))
    return '\n'.join(map(row, range(self.height))) +  '\n'

In [36]:
def random_player(game, state):
  return random.choice(list(game.actions(state)))
def player(sample_algorithm):
  return lambda game, state: sample_algorithm(game, state)[1]
  play_game(tictactoe(), dict(X=random_player, O=player(alphabeta_search)), verbose=True).utility